In [1]:
import os
import geopandas as gpd
import networkx as nx
import pyogrio
import numpy as np
import matplotlib.pyplot as plt
from xrspatial.zonal import stats, crosstab
import rioxarray

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
data_path = 'high_res_data/NHDPLUS_H_1710_HU4_GPKG.gpkg'
# data_path = 'high_res_data/NHD_H_Oregon_State_GDB.gdb'
layers = pyogrio.list_layers(data_path)
layers

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\pyogrio\core.py:117: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


array([['NHDArea', 'MultiPolygon Z'],
       ['NHDFlowline', 'MultiLineString Z'],
       ['NHDLine', 'MultiLineString Z'],
       ['NHDPlusBurnLineEvent', 'MultiLineString'],
       ['NHDPlusBurnWaterbody', 'MultiPolygon'],
       ['NHDPlusCatchment', 'MultiPolygon'],
       ['NHDPlusLandSea', 'MultiPolygon'],
       ['NHDPlusSink', 'Point'],
       ['NHDPlusWall', 'MultiLineString'],
       ['NHDPoint', 'Point Z'],
       ['NHDWaterbody', 'MultiPolygon Z'],
       ['NWISDrainageArea', 'MultiPolygon'],
       ['NWISDrainageLine', 'MultiLineString'],
       ['NonContributingDrainageArea', 'MultiPolygon'],
       ['NonContributingDrainageLine', 'MultiLineString'],
       ['WBDHU10', 'MultiPolygon'],
       ['WBDHU12', 'MultiPolygon'],
       ['WBDHU14', 'MultiPolygon'],
       ['WBDHU16', 'MultiPolygon'],
       ['WBDHU2', 'MultiPolygon'],
       ['WBDHU4', 'MultiPolygon'],
       ['WBDHU6', 'MultiPolygon'],
       ['WBDHU8', 'MultiPolygon'],
       ['WBDLine', 'MultiLineString'],
     

In [3]:

flow = gpd.read_file(data_path, layer='NHDPlusFlow', engine='pyogrio') 
fls = gpd.read_file(data_path, layer='NHDFlowline', engine='pyogrio')
vaa = gpd.read_file(data_path, layer='NHDPlusFlowlineVAA', engine='pyogrio')
cat = gpd.read_file(data_path, layer='NHDPlusCatchment', engine='pyogrio')

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(


In [4]:
flow.head()

,FromNHDPID,ToNHDPID,NodeNumber,DeltaLevel,Direction,GapDistKm,HasGeo,FromVPUID,ToVPUID,FromPermID,ToPermID,HasGeo_description
0,0.0,5.500010e+13,5.500010e+13,0,712,0.0,1,None,1710,0,160860908,Yes
1,0.0,5.500010e+13,5.500010e+13,0,712,0.0,1,None,1710,0,124302653,Yes
2,0.0,5.500010e+13,5.500010e+13,0,712,0.0,1,None,1710,0,160860968,Yes
3,0.0,5.500010e+13,5.500010e+13,0,712,0.0,1,None,1710,0,{327F989D-1A01-427B-9023-EAD03642B0E0},Yes
4,0.0,5.500010e+13,5.500010e+13,0,712,0.0,1,None,1710,0,161862876,Yes


In [5]:
fls.head()

,Permanent_Identifier,FDate,Resolution,GNIS_ID,GNIS_Name,LengthKM,ReachCode,FlowDir,WBArea_Permanent_Identifier,FType,...,NHDPlusID,VPUID,Enabled,Resolution_description,FlowDir_description,MainPath_description,InNetwork_description,VisibilityFilter_description,fcode_description,geometry
0,165135834,2018-04-25 00:00:00+00:00,2,None,None,0.190647,17100306020353,1,None,460,...,5.500010e+13,1710,1,High,WithDigitized,Unspecified,Yes,Unspecified,Stream/River: Hydrographic Category = Intermit...,MULTILINESTRING Z ((-124.22646 42.86454 0.0000...
1,166018914,2018-04-25 00:00:00+00:00,2,01117415,Beaver Creek,0.476448,17100305000373,1,None,460,...,5.500010e+13,1710,1,High,WithDigitized,Unspecified,Yes,Unspecified,Stream/River: Hydrographic Category = Perennial,MULTILINESTRING Z ((-124.21603 43.23285 0.0000...
2,{BE4D9DCA-9B54-46C2-8915-3FBF5814594F},2018-04-24 00:00:00+00:00,2,None,None,0.060162,17100305009600,1,None,460,...,5.500010e+13,1710,1,High,WithDigitized,Unspecified,Yes,Unspecified,Stream/River: Hydrographic Category = Intermit...,MULTILINESTRING Z ((-124.21223 43.24349 0.0000...
3,{182F9DA5-74A0-4853-ABBB-FF847A3DDEE2},2018-04-24 00:00:00+00:00,2,None,None,0.074088,17100305044921,1,None,460,...,5.500010e+13,1710,1,High,WithDigitized,Unspecified,Yes,Unspecified,Stream/River: Hydrographic Category = Intermit...,MULTILINESTRING Z ((-124.21117 43.23736 0.0000...
4,{4D4340AB-73E6-4283-97E8-CEF2D87A058D},2018-04-24 00:00:00+00:00,2,None,None,0.277113,17100305044914,1,None,460,...,5.500010e+13,1710,1,High,WithDigitized,Unspecified,Yes,Unspecified,Stream/River: Hydrographic Category = Intermit...,MULTILINESTRING Z ((-124.21085 43.24224 0.0000...


In [6]:
vaa.head()

,NHDPlusID,StreamLeve,StreamOrde,StreamCalc,FromNode,ToNode,HydroSeq,LevelPathI,PathLength,TerminalPa,...,VPUID,Divergence_description,StartFlag_description,TerminalFl_description,RtnDiv_description,VPUIn_description,VPUOut_description,ElevFixed_description,HWType_description,StatusFlag_description
0,5.500010e+13,3,1,1,5.500010e+13,5.500010e+13,5.500010e+13,5.500010e+13,3.220125,5.500010e+13,...,1710,Feature is not part of a divergence,Yes,No,No,No,No,No,Real Headwater,Add
1,5.500010e+13,3,2,2,5.500010e+13,5.500010e+13,5.500010e+13,5.500010e+13,4.007858,5.500010e+13,...,1710,Feature is not part of a divergence,No,No,No,No,No,No,Undefined,Add
2,5.500010e+13,6,1,1,5.500010e+13,5.500010e+13,5.500010e+13,5.500010e+13,24.775808,5.500010e+13,...,1710,Feature is not part of a divergence,Yes,No,No,No,No,Yes,Real Headwater,Add
3,5.500010e+13,3,1,1,5.500010e+13,5.500010e+13,5.500010e+13,5.500010e+13,29.674186,5.500010e+13,...,1710,Feature is not part of a divergence,Yes,No,No,No,No,No,Real Headwater,Add
4,5.500010e+13,3,1,1,5.500010e+13,5.500010e+13,5.500010e+13,5.500010e+13,3.747671,5.500010e+13,...,1710,Feature is not part of a divergence,Yes,No,No,No,No,No,Real Headwater,Add


In [7]:
cat.head()

,NHDPlusID,SourceFC,GridCode,AreaSqKm,VPUID,SHAPE_Length,SHAPE_Area,geometry
0,5.500010e+13,NHDPlusBurnLineEvent,587654,0.0389,1710,0.013786,0.000005,"MULTIPOLYGON (((-124.62939 48.21301, -124.6299..."
1,5.500010e+13,NHDPlusBurnLineEvent,26064,0.2719,1710,0.055632,0.000033,"MULTIPOLYGON (((-124.61768 48.21549, -124.6178..."
2,5.500010e+13,NHDPlusBurnLineEvent,3899,0.0507,1710,0.015380,0.000006,"MULTIPOLYGON (((-124.57229 48.21700, -124.5725..."
3,5.500010e+13,NHDPlusBurnLineEvent,584956,0.0403,1710,0.011203,0.000005,"MULTIPOLYGON (((-124.52669 48.22661, -124.5267..."
4,5.500010e+13,NHDPlusBurnLineEvent,22600,0.4650,1710,0.039445,0.000056,"MULTIPOLYGON (((-124.67436 48.19532, -124.6744..."


In [8]:
flow = flow[~((flow.FromPermID == '0') | (flow.ToPermID == '0'))] # same as current numpy process
coasties = fls.loc[fls.FType == 566,"Permanent_Identifier"]        # same as current numpy process
flow = flow[~flow.FromPermID.isin(coasties)]                 # same as current numpy process
fls.to_crs("epsg:5070", inplace=True)
fls.geometry = fls.geometry.centroid

In [9]:
flow = flow.loc[:, ['FromPermID','ToPermID']]
G = nx.DiGraph()
G.add_edges_from(flow.to_records(index=False))
flow.head()

,FromPermID,ToPermID
244650,167113583,161454674
346342,160880152,124308361
346343,124314913,124307252
346344,160879764,160880043
346345,160879690,160880260


In [26]:
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=False, node_color='lightblue', edge_color='gray')
plt.show()

KeyboardInterrupt: 

In [ ]:

permID = '129459027' # for 1710 gpkg
upstream = nx.bfs_tree(G, permID, reverse=True) # reverse means upstream for a particular PermID
print(len(upstream))
print(list(upstream)[:13])       # the first 13 PermIDs upstream in a list
print(np.array(upstream)[:13])   # or in an array
top = np.array(upstream)[-1] 

In [ ]:
len(top)
len(upstream)

In [ ]:
downstream = nx.bfs_tree(G, top)
print(len(downstream))
print(np.array(downstream)[:13])

In [ ]:

medomak = nx.bfs_tree(G, permID, reverse=True) # snake convergence: 23475172
ups = fls.loc[fls.Permanent_Identifier.isin(list(medomak))].set_index("Permanent_Identifier")
pos = ups.apply(lambda g: (g.geometry.x, g.geometry.y), axis=1).to_dict()
nx.draw_networkx(nx.DiGraph.reverse(medomak), pos, node_size=147, node_color='green',
                 font_size=18, with_labels=False, width=2)
plt.title('Medomak River Watershed', fontsize = 40)
plt.rcParams["figure.figsize"] = (40,20)
plt.show()

In [ ]:
medomak_down = nx.bfs_tree(G, permID) 
ups = fls.loc[fls.Permanent_Identifier.isin(list(medomak_down))].set_index("Permanent_Identifier")
pos = ups.apply(lambda g: (g.geometry.x, g.geometry.y), axis=1).to_dict()
nx.draw_networkx(medomak_down, pos, node_size=147, node_color='green',
                 font_size=18, with_labels=False, width=2)
plt.suptitle('Downstream View from HeadWater', fontsize = 40)
plt.title("the last node out on the right is a 'Coastline' PermID", fontsize=25)
plt.rcParams["figure.figsize"] = (40,20)
plt.show()
fls.loc[fls.Permanent_Identifier == np.array(medomak_down)[-1]]

In [ ]:

cat['NHDPlusID'] = cat['NHDPlusID'].astype(str)

fls['NHDPlusID'] = fls['NHDPlusID'].astype(str)
temp=fls[['NHDPlusID','Permanent_Identifier']]
cats = cat.merge(temp, on='NHDPlusID')
# cats['row_num'] = cats.reset_index().index
cats = cats[['Permanent_Identifier','NHDPlusID','AreaSqKm','geometry']]
cats.head()

In [ ]:
cats.dtypes

In [38]:
cats['NHDPlusID'] = cats['NHDPlusID'].astype(str)
cats['Permanent_Identifier'] = cats['Permanent_Identifier'].astype(str)

In [ ]:

clay = "high_res_data/NHDPLUS_H_1710_HU4_RASTER.7z"
clay_arr = rioxarray.open_rasterio(clay)
clay_arr.data.shape

In [ ]:
zones_geojson = cats["geometry"].to_crs("epsg:5070").to_json()
zones_da = rioxarray.open_rasterio(zones_geojson)
zones_da.data

In [ ]:
cats = cats.to_crs(5070)
clay_stats = stats(clay_arr[0], cats_arr, stats_funcs=["count", "mean", "sum"], return_type='pandas.DataFrame')
clay_stats.head()

In [ ]:
H = G.subgraph(list(upstream))
# np.array(list(nx.get_node_attributes(H, "clay").values())).mean()
clay_ws = np.array(list(nx.get_node_attributes(H, "clay").values())).mean()
print('ClayWs is ' + '%.2f'%(clay_ws))